
# Image Compression Tool — JPEG-like (8×8 DCT + Quantization)
**What this notebook contains (ready to run):**

- Short theory about JPEG pipeline (DCT, quantization, YCrCb conversion).  
- Implementation of a simplified JPEG-like compressor (no entropy coding).  
- Adjustable quality parameter (1–100) to observe trade-offs between compression and quality.  
- Metrics: PSNR and SSIM.  
- Visualizations: original, compressed, and difference images.  

> This notebook is designed for demonstration and learning. It is perfect as a Jupyter demo during your mini-project presentation.



## 1. Setup — Install & Imports
Run the cell below to install any missing packages (only needed if packages aren't present in your environment).
The code that follows uses: `numpy`, `opencv-python`, `scikit-image`, `matplotlib`, `pandas`.


In [4]:
# Uncomment and run the following line if you need to install packages (skip if already installed).
# !pip install numpy opencv-python scikit-image matplotlib pandas

import numpy as np
import cv2
from skimage import data, img_as_float
from skimage.metrics import structural_similarity as ssim
import matplotlib.pyplot as plt
import pandas as pd
from math import log10, sqrt
import os
%matplotlib inline

# Output directory
out_dir = "/home/vaishnav/Documents/DIP MINI Project"
os.makedirs(out_dir, exist_ok=True)
print("Outputs will be saved to:", out_dir)


Outputs will be saved to: /home/vaishnav/Documents/DIP MINI Project


## 2. Brief Theory (memorize-friendly)

**2D DCT (8×8 block):**

\[
F(u,v) = \frac{1}{4}C(u)C(v)\sum_{x=0}^{7}\sum_{y=0}^{7} f(x,y)\cos\!\left[\frac{(2x+1)u\pi}{16}\right]\cos\!\left[\frac{(2y+1)v\pi}{16}\right]
\]

with \(C(0)=\frac{1}{\sqrt{2}}\), \(C(k)=1\) for \(k>0\).

**Quantization:**  
\[
F_q(u,v) = \text{round}\left(\frac{F(u,v)}{Q(u,v)}\right)
\]

**Inverse:**  
\[
F'(u,v) = F_q(u,v) \times Q(u,v)
\]

**PSNR:**  
\[
\text{PSNR} = 20\log_{10}\left(\frac{MAX}{\sqrt{\text{MSE}}}\right),\quad MAX=255
\]

**Why YCrCb?** Human vision is more sensitive to luminance (Y). Chrominance channels (Cr,Cb) can be quantized/heavily compressed with less perceptual loss.



## 3. Helper functions — block processing, DCT/IDCT, quantization matrices
The next cell implements helper functions used throughout the notebook.


In [6]:

# Standard JPEG quantization matrices (luminance & chrominance)
QY = np.array([
 [16,11,10,16,24,40,51,61],
 [12,12,14,19,26,58,60,55],
 [14,13,16,24,40,57,69,56],
 [14,17,22,29,51,87,80,62],
 [18,22,37,56,68,109,103,77],
 [24,35,55,64,81,104,113,92],
 [49,64,78,87,103,121,120,101],
 [72,92,95,98,112,100,103,99]
], dtype=np.float32)

QC = np.array([
 [17,18,24,47,99,99,99,99],
 [18,21,26,66,99,99,99,99],
 [24,26,56,99,99,99,99,99],
 [47,66,99,99,99,99,99,99],
 [99,99,99,99,99,99,99,99],
 [99,99,99,99,99,99,99,99],
 [99,99,99,99,99,99,99,99],
 [99,99,99,99,99,99,99,99]
], dtype=np.float32)

def block_process(channel, block_size, func):
    h, w = channel.shape
    h_pad = (block_size - (h % block_size)) % block_size
    w_pad = (block_size - (w % block_size)) % block_size
    padded = np.pad(channel, ((0,h_pad),(0,w_pad)), mode='edge')
    out = np.zeros_like(padded)
    for i in range(0, padded.shape[0], block_size):
        for j in range(0, padded.shape[1], block_size):
            block = padded[i:i+block_size, j:j+block_size]
            out[i:i+block_size, j:j+block_size] = func(block)
    return out[:h, :w]

def dct2(block):
    return cv2.dct(block.astype(np.float32))

def idct2(block):
    return cv2.idct(block.astype(np.float32))

def scale_quant_matrix(Q, quality):
    # quality in 1..100 like JPEG
    if quality < 1: quality = 1
    if quality > 100: quality = 100
    if quality < 50:
        scale = 5000 / quality
    else:
        scale = 200 - 2*quality
    Qs = ((Q * scale) + 50) // 100
    Qs[Qs == 0] = 1
    return Qs.astype(np.float32)



## 4. Compression / Decompression function
This cell implements `compress_decompress(img_rgb, quality)` which:
- Converts RGB → YCrCb
- Processes each channel in 8×8 blocks: shift by -128, DCT, quantize, dequantize, inverse DCT
- Reconstructs RGB image


In [7]:

def compress_decompress(img_rgb, quality=50):
    # Convert to YCrCb (OpenCV expects BGR input, so convert)
    img_bgr = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR)
    img_ycrcb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2YCrCb).astype(np.float32)
    channels = cv2.split(img_ycrcb)
    # Scale quant matrices
    QY_s = scale_quant_matrix(QY, quality)
    QC_s = scale_quant_matrix(QC, quality)
    out_channels = []
    for idx, ch in enumerate(channels):
        ch_shifted = ch - 128.0
        Q = QY_s if idx == 0 else QC_s
        def proc(block):
            B = dct2(block)
            Bq = np.round(B / Q)
            Bd = Bq * Q
            rec = idct2(Bd)
            return rec
        rec = block_process(ch_shifted, 8, proc)
        rec = rec + 128.0
        rec = np.clip(rec, 0, 255)
        out_channels.append(rec)
    ycrcb_rec = cv2.merge(out_channels).astype(np.uint8)
    bgr_rec = cv2.cvtColor(ycrcb_rec, cv2.COLOR_YCrCb2BGR)
    rgb_rec = cv2.cvtColor(bgr_rec, cv2.COLOR_BGR2RGB)
    return rgb_rec



## 5. Metrics — PSNR and SSIM
Use PSNR (numeric) and SSIM (perceptual) to compare original and reconstructed images.


In [8]:

def psnr(original, compressed):
    original = original.astype(np.float32)
    compressed = compressed.astype(np.float32)
    mse = np.mean((original - compressed) ** 2)
    if mse == 0:
        return float('inf')
    PIXEL_MAX = 255.0
    return 20 * log10(PIXEL_MAX / sqrt(mse))

def compute_metrics(orig, rec):
    p = psnr(orig, rec)
    s = ssim(orig, rec, multichannel=True, data_range=255)
    return p, s



## 6. Demo — Run compression for multiple quality values and visualize results
The default demo uses `skimage.data.astronaut()`. Replace with your own image by loading it (e.g., `cv2.imread`).

Run the next cell to produce images and metrics for qualities: 10, 30, 50, 70, 90.


In [10]:
# Corrected compute_metrics + metrics loop (robust to skimage versions)

def psnr(original, compressed):
    original = original.astype(np.float32)
    compressed = compressed.astype(np.float32)
    mse = np.mean((original - compressed) ** 2)
    if mse == 0:
        return float('inf')
    PIXEL_MAX = 255.0
    return 20 * log10(PIXEL_MAX / sqrt(mse))

def compute_metrics(orig, rec):
    """Compute PSNR and SSIM in a way that works with different skimage versions."""
    p = psnr(orig, rec)
    # Try new API first (channel_axis). If not available, fall back to multichannel.
    try:
        s = ssim(orig, rec, channel_axis=2, data_range=255)
    except TypeError:
        # older versions of skimage expect multichannel=True
        s = ssim(orig, rec, multichannel=True, data_range=255)
    except ValueError as e:
        # If we still get a win_size error because image is tiny, try a smaller win_size
        # (choose the largest odd win_size <= min(height, width))
        min_side = min(orig.shape[0], orig.shape[1])
        win = 7
        # pick largest odd <= min_side
        if min_side < 7:
            win = min_side if (min_side % 2 == 1) else (min_side - 1)
            if win < 3:
                win = 3  # fallback minimum
        try:
            s = ssim(orig, rec, channel_axis=2, data_range=255, win_size=win)
        except TypeError:
            s = ssim(orig, rec, multichannel=True, data_range=255, win_size=win)
    return p, s

# Run metrics collection (same demo results list you already have)
metrics = []
for q, rec in results:
    path = os.path.join(out_dir, f"astronaut_Q{q}.png")
    cv2.imwrite(path, cv2.cvtColor(rec, cv2.COLOR_RGB2BGR))
    p, s = compute_metrics(orig, rec)
    metrics.append({"quality": q, "psnr": round(p,2), "ssim": round(s,4), "path": path})

df = pd.DataFrame(metrics)
csv_path = os.path.join(out_dir, "metrics_notebook.csv")
df.to_csv(csv_path, index=False)
print("Saved comparison image to:", fig_path)
print("Saved metrics CSV to:", csv_path)
df


Saved comparison image to: /home/vaishnav/Documents/DIP MINI Project/comparison_grid_notebook.png
Saved metrics CSV to: /home/vaishnav/Documents/DIP MINI Project/metrics_notebook.csv


,quality,psnr,ssim,path
0,10,27.28,0.8143,/home/vaishnav/Documents/DIP MINI Project/astr...
1,30,31.37,0.9093,/home/vaishnav/Documents/DIP MINI Project/astr...
2,50,33.03,0.9318,/home/vaishnav/Documents/DIP MINI Project/astr...
3,70,34.69,0.9472,/home/vaishnav/Documents/DIP MINI Project/astr...
4,90,38.28,0.9678,/home/vaishnav/Documents/DIP MINI Project/astr...
